In [ ]:
# In this file the process to compute the reservoir state derivative of an Echo State Netowrk is
# defined as a cell for a tensorflow Recurrent Neural Netowrk

# to pass to py_func
def np_eigenvals(x):
    return np.linalg.eigvals(x).astype('complex128')

# Definition of the EchoState NN as a child of RNNCell of tensorflow
# It generates the states of the reservoir; there is no output matrix
class EchoStateRNNCelldrdt(tf.keras.layers.AbstractRNNCell):

    def __init__(self, num_units, num_inputs=1, decay=0.1, rho=0.6,
                 sparseness=0.0,
                 sigma_in=1.0,
                 b_in=1.0,
                 rng=None,
                 activation=None,
                 reuse = False,
                 win=None,
                 wecho=None):
        """
        Args:
            num_units: int, Number of units in the ESN cell.
            num_inputs: int, The number of inputs to the RNN cell.
            decay: float, Decay/leaking of the ODE of each unit.
            rho: float, Target spectral radius 1.
            sparseness: float [0,1], sparseness of the inner weight matrix.
            rng: np.random.RandomState, random number generator. (to be able to have always the same random matrix...)
            activation: Nonlinearity to use.
            reuse: if reusing existing matrix
            win: input weights matrix
            wecho: echo state matrix
        """

        # Basic RNNCell initialization (see tensorflow documentation)
        super(EchoStateRNNCelldrdt, self).__init__() # use the initializer of RNNCell (i.e. defines a bunch of standard variables)
        
        # hyperparameters
        self._num_units  = num_units
        self._activation = activation
        self._num_inputs = num_inputs
        self.decay = decay
        self.rho = rho
        self.sparseness = sparseness
        self.sigma_in = sigma_in
        self.b_in  = np.reshape(b_in, [1,1])
        
        # Random number generator initialization
        self.rng = rng
        if rng is None:
            self.rng = np.random.RandomState()
        
        # build initializers for tensorflow variables
        if (reuse == False):
            self.win = self.buildInputMatrix()
            self.wecho = self.buildEchoMatrix()
        else: #in case they are passed as an argument
            self.win = win.astype('float64')
            self.wecho = wecho.astype('float64')
        
        # convert the weight to tf variable
        self.Win   = tf.Variable(self.win, name='Win', trainable=False) 
        self.Wecho = tf.Variable(self.wecho, name='Wecho', trainable=False)

        self.setEchoStateProperty()

    @property
    def state_size(self):
        return self._num_units*2

    @property
    def output_size(self):
        return self._num_units*2
    
    # function that has to be implemented in the tensorflow framework
    # return the output and new state of the RNN for given inputs and current state of the RNN
    # state and output coincide in the reservoir
    def call(self, inputs, state):
        """ Reservoir state and reservoir state evolution: 
            x(i+1) = f(W*inp(i) + U*g(x(i))).
            dx_dt(i+1) = dx(i+1)_d(inp(i))*d(inp(i))_dt + dx(i+1)_d(x(i))*d(x(i))_dt
        """
        
        # separating input and its derivative
        x     = inputs[:,:self._num_inputs]
        x_der = inputs[:,self._num_inputs:]
        # separating state and its derivative
        s     = state[0][:,:self._num_units]
        s_der = state[0][:,self._num_units:]
        
        #state at next time step
        new_state = self._activation(
                    tf.matmul(tf.concat([x,self.b_in],axis=1),self.Win) +
                    tf.matmul(s, self.Wecho))
        
        # initialize state derivative at next time step
        deriv     = tf.TensorArray(
                    tf.float64, self._num_units, dynamic_size=False, element_shape=[1,1])
        
        # Compute the derivative: dr2_dt = dr2_dx * dx_dt + dr2_dr1 * dr1_dt
        # dx_dt is x_der and dr1_dt is s_der
        # need to use a for loop with tf.gradients since tf.jacobian does not work in graph mode for RNN as of now (even with tape.gradient)
        # k = 0
        for j in tf.range(self._num_units):
            drj_dt   = tf.reshape(tf.reduce_sum(tf.multiply(tf.gradients(new_state[0,j],x)[0],
                                                                  x_der)), [1,1]) + \
                       tf.reshape(tf.reduce_sum(tf.multiply(tf.gradients(new_state[0,j],s)[0],
                                                                  s_der)), [1,1])
            deriv    = deriv.write(j,drj_dt)
#             k       += 1
            

        new_state = tf.concat([new_state, tf.reshape(deriv.stack()[:,0],[1,self._num_units])],
                              axis=1)
        
        return new_state, new_state   
    
    def setEchoStateProperty(self):
        """ optimize U to obtain alpha-improved echo-state property """
        self.Wecho = self.normalizeEchoStateWeights(self.Wecho)
        

    # construct the Win matrix (dimension num_inputs x num_units)
    def buildInputMatrix(self):
        """            
            Returns:
            
            Matrix representing the 
            input weights to an ESN    
        """  

        # Each unit is connected randomly to a given input with a weight from a 
        # uniform distribution between +- sigma_in (input scaling)
        # +1 for added input bias in the input matrix
        W = np.zeros((self._num_inputs+1,self._num_units))
        for i in range(self._num_units):
            W[self.rng.randint(0,self._num_inputs+1),i] = \
            self.rng.uniform(-self.sigma_in,self.sigma_in)
                    
        return W.astype('float64')
    
    def getInputMatrix(self):
        return self.win
    
    def buildEchoMatrix(self):
        """            
            Returns:
            
            Matrix representing the 
            inner weights to an ESN    
        """    
        
        # Build random matrix from uniform distribution
        W = self.rng.uniform(-1.0,1.0, [self._num_units, self._num_units]).astype("float64") * \
                (self.rng.rand(self._num_units, self._num_units) < (1. - self.sparseness) ) # trick to add zeros to have the sparseness required
        return W
    
    def normalizeEchoStateWeights(self, W):
        # Sets the spectral radius rho

        eigvals = tf.py_function(np_eigenvals, [W], tf.complex128)
        W = W / tf.reduce_max(tf.abs(eigvals))*self.rho #imposes spectral radius

        return W
    
    def getEchoMatrix(self):
        return self.wecho

In [1]:
# # NAKD
# import numpy as np
# import tensorflow as tf

# import h5py

# # to pass to py_func
# def np_eigenvals(x):
#     return np.linalg.eigvals(x).astype('complex128')

# # Definition of the EchoState NN as a child of RNNCell of tensorflow
# class EchoStateRNNCelldrdt(tf.keras.layers.AbstractRNNCell):

#     def __init__(self, num_units, num_inputs=1, decay=0.1, rho=0.6,
#                  sparseness=0.0,
#                  sigma_in=1.0,
#                  b_in=1.0,
#                  rng=None,
#                  activation=None,
#                  reuse = False,
#                  win=None,
#                  wecho=None):
#         """
#         Args:
#             num_units: int, Number of units in the ESN cell.
#             num_inputs: int, The number of inputs to the RNN cell.
#             decay: float, Decay/leaking of the ODE of each unit.
#             rho: float, Target spectral radius 1.
#             sparseness: float [0,1], sparseness of the inner weight matrix.
#             rng: np.random.RandomState, random number generator. (to be able to have always the same random matrix...)
#             activation: Nonlinearity to use.
#             reuse: if reusing existing matrix
#             win: input weights matrix
#             wecho: echo state matrix
#         """

#         # Basic RNNCell initialization (see tensorflow documentation)
#         super(EchoStateRNNCelldrdt, self).__init__() # use the initializer of RNNCell (i.e. defines a bunch of standard variables)
        
#         # fixed variables
#         self._num_units  = num_units
#         self._activation = activation
#         self._num_inputs = num_inputs
        
#         # variables that potentially can be changed/optimized (for future version)
#         self.decay = decay
#         self.rho = rho # rho has to be <1. to ensure the echo state property (see [2])
#         self.sparseness = sparseness
#         self.sigma_in = sigma_in
#         self.b_in  = np.reshape(b_in, [1,1])
        
#         # Random number generator initialization
#         self.rng = rng
#         if rng is None:
#             self.rng = np.random.RandomState()
        
#         # build initializers for tensorflow variables
#         if (reuse == False):
#             self.win = self.buildInputMatrix()
#             self.wecho = self.buildEchoMatrix()
#         else:
#             self.win = win.astype('float64')
#             self.wecho = wecho.astype('float64')
        
#         # convert the weight to tf variable
#         self.Win   = tf.Variable(self.win, name='Win', trainable=False) 
#         self.Wecho = tf.Variable(self.wecho, name='Wecho', trainable=False)

#         self.setEchoStateProperty()

#     @property
#     def state_size(self):
#         return self._num_units*2

#     @property
#     def output_size(self):
#         return self._num_units*2


#     # function that has to be implemented in the tensorflow framework
#     # receives input and previous state and returns state and output (which are the same for the ESN without Wout)
#     def call(self, inputs, state):
#         """ Compute the reservoir state and its derivative """

#         # separating input and its derivative
#         x     = inputs[:,:self._num_inputs]
#         x_der = inputs[:,self._num_inputs:]
#         # separating state and its derivative
#         s     = state[0][:,:self._num_units]
#         s_der = state[0][:,self._num_units:]
        
#         #state at next time step
#         new_state = self._activation(
#                     tf.matmul(tf.concat([x,self.b_in],axis=1),self.Win) +
#                     tf.matmul(s, self.Wecho))
        
#         # initialize state derivative at next time step
#         deriv     = tf.TensorArray(
#                     tf.float64, self._num_units, dynamic_size=False, element_shape=[1,1])
        
#         # Compute the derivative: dr2_dt = dr2_dx * dx_dt + dr2_dr1 * dr1_dt
#         # dx_dt is x_der and dr1_dt is s_der
#         # need to use a for loop with tf.gradients since tf.jacobian does not work in graph mode for RNN as of now (even with tape.gradient)
#         # k = 0
#         for j in tf.range(self._num_units):
#             drj_dt   = tf.reshape(tf.reduce_sum(tf.multiply(tf.gradients(new_state[0,j],x)[0],
#                                                                   x_der)), [1,1]) + \
#                        tf.reshape(tf.reduce_sum(tf.multiply(tf.gradients(new_state[0,j],s)[0],
#                                                                   s_der)), [1,1])
#             deriv    = deriv.write(j,drj_dt)
# #             k       += 1
            

#         new_state = tf.concat([new_state, tf.reshape(deriv.stack()[:,0],[1,self._num_units])],
#                               axis=1)
        
#         return new_state, new_state   
    
#     def setEchoStateProperty(self):
#         """ optimize U to obtain alpha-improved echo-state property """
#         # I know it's stupid for the time being but it is a placeholder for future treatment of the matrix
#         # (potential meta-optimization and other)
#         self.Wecho = self.normalizeEchoStateWeights(self.Wecho)
        

#     # construct the Win matrix (dimension num_inputs x num_units)
#     def buildInputMatrix(self):
#         """            
#             Returns:
            
#             Matrix representing the 
#             input weights to an ESN    
#         """  

#         # Input weight matrix initializer according to [3,4]
#         # Each unit is connected randomly to a given input with a weight from a uniform distribution
        
#         # without bias at the input
#         #W = np.zeros((self._num_inputs,self._num_units))
#         #for i in range(self._num_units):
#             #W[self.rng.randint(0,self._num_inputs),i] = self.rng.uniform(-self.sigma_in,self.sigma_in)
        
#         # Added bias in the input matrix
#         W = np.zeros((self._num_inputs+1,self._num_units))
#         for i in range(self._num_units):
#             W[self.rng.randint(0,self._num_inputs+1),i] = \
#             self.rng.uniform(-self.sigma_in,self.sigma_in)
            
#         # Dense input weigth [input] as in [1,2]
#         # Input weigth matrix [input]
#         #W = self.rng.uniform(-self.sigma_in, self.sigma_in, [self.num_inputs, self._num_units]).astype("float64")
        
#         # Dense input weigth [bias, input] (as in [1,2])
#         #W = self.rng.uniform(-self.sigma_in, self.sigma_in, [self.num_inputs+1, self._num_units]).astype("float64")
        
#         return W.astype('float64')
    
#     def getInputMatrix(self):
#         return self.win
    
#     def buildEchoMatrix(self):
#         """            
#             Returns:
            
#             A 1-D tensor representing the 
#             inner weights to an ESN (to be optimized)        
#         """    

#         # Inner weight tensor initializer
#         # 1) Build random matrix from normal distribution between [0.,1.]
#         #W = self.rng.randn(self._num_units, self._num_units).astype("float64") * \
#                 #(self.rng.rand(self._num_units, self._num_units) < (1. - self.sparseness) )
        
#         # 2) Build random matrix from uniform distribution
#         W = self.rng.uniform(-1.0,1.0, [self._num_units, self._num_units]).astype("float64") * \
#                 (self.rng.rand(self._num_units, self._num_units) < (1. - self.sparseness) ) # trick to add zeros to have the sparseness required
#         return W
    
#     def normalizeEchoStateWeights(self, W):
#         # Normalize to spectral radius rho

#         eigvals = tf.py_function(np_eigenvals, [W], tf.complex128)
#         W = W / tf.reduce_max(tf.abs(eigvals))*self.rho # sufficient conditions to ensure that the spectral radius is rho

#         return W
    
#     def getEchoMatrix(self):
#         return self.wecho

# #     def save_ESN(self,fln,Wout=None,CurrState=None,lmb=None):
        
# #         hf = h5py.File(fln,'w')
        
# #         hf.create_dataset('Win',data=self.getInputMatrix() )
# #         hf.create_dataset('Wecho',data=self.getEchoMatrix() )
# #         hf.create_dataset('Wout',data=Wout)
        
# #         hf.create_dataset('rho',data=self.rho)
# #         hf.create_dataset('num_inputs',data=self._num_inputs)
# #         hf.create_dataset('num_units',data=self._num_units)
# #         hf.create_dataset('sigma_in',data=self.sigma_in)
# #         hf.create_dataset('sparseness',data=self.sparseness)
# #         hf.create_dataset('decay',data=self.decay)
# #         hf.create_dataset('lmb',data=lmb)
# #         hf.create_dataset('state',data=CurrState)
        
# #         hf.close()